<a href="https://colab.research.google.com/github/DamodaraBarbosa/challenge_data_science_alura_week_1/blob/main/Challenge_Data_Science_Alura_Semana_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando dependências do Spark

In [128]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

Importando OS

In [129]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

Importando o Spark Session

In [130]:
import findspark
findspark.init()

In [131]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

In [132]:
spark

# Carregando dados no dataframe

In [133]:
data = spark.read.parquet('/content/drive/MyDrive/Alura/Challenge Data Science/part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet')

In [134]:
data.show(10)

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|       Usado| Apartamento|Residencial|   1|            Realengo|  Zona Oeste|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residencial|   0|               Irajá|  Zona Norte|       170|   0|Vend

# Seleção de features

In [135]:
# para selecionar as features utiliza-se o método distinct() a fim de obter os valores únicos para cada coluna do dataframe, num primeiro momento suspeita-se
# das colunas tipo_anuncio, tipo_uso e tipo.

### Coluna tipo_anuncio

In [136]:
data.select('tipo_anuncio').distinct().show()

+------------+
|tipo_anuncio|
+------------+
|       Usado|
+------------+



In [137]:
# todos os dados da coluna tipo_anuncio são iguais: "Usado"

### Coluna tipo_uso

In [138]:
data.select('tipo_uso').distinct().show()

+-----------+
|   tipo_uso|
+-----------+
|Residencial|
+-----------+



In [139]:
# todos os dados da coluna tipo_uso são iguais: "Residencial"

### Coluna tipo_unidade

In [140]:
data.select('tipo_unidade').distinct().show()

+------------+
|tipo_unidade|
+------------+
| Apartamento|
+------------+



In [141]:
# todos os dados da coluna tipo_unidade são iguais: "Apartamento"

### Coluna tipo

In [142]:
data.select('tipo').distinct().show()

+-----+
| tipo|
+-----+
|Venda|
+-----+



In [143]:
# todos os dados da coluna tipo são iguais: "Venda"

## Drop das colunas tipo_anuncio, tipo_uso, tipo_unidade e tipo


In [144]:
# como as colunas citadas acima possuem apenas uma única informação, podem ser eliminadas do dataframe a fim de deixar apenas dados mais significativos:

data = data.drop('tipo_anuncio')
data = data.drop('tipo_uso')
data = data.drop('tipo_unidade')
data = data.drop('tipo')
data.show(5)

+--------------------+-----+----------+---------+---------+--------------------+-------+------+----+------------+------------+----------+----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|vaga|      bairro|        zona|condominio|iptu| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+----+------------+------------+----------+----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|   1|    Realengo|  Zona Oeste|       285|null| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|   0|       Irajá|  Zona Norte|       170|   0|110000|
|1fa1c1e5-e98c-433...|    4|      null|       55|        1|                  []|      2|     0|   1|      Cosmos|  Zona Oeste|      null|null|115000|
|a6ab01ae-3d40-40e...|    2|      null|       55|        1|                  []|      2|     0|   0|

## Colunas area_total e area_util

In [145]:
# contando os valores da coluna area_total:

data.select('area_total').groupBy('area_total').count().show()

+----------+-----+
|area_total|count|
+----------+-----+
|       296|   31|
|       691|    1|
|       675|    1|
|       125|  253|
|       800|    5|
|   5698720|    1|
|        51|  206|
|       124|  159|
|         7|    2|
|       447|    1|
|       307|   25|
|     17583|    3|
|      1695|    1|
|      1500|    1|
|       205|   44|
|       169|   75|
|      7402|    4|
|       334|    6|
|       544|    1|
|       272|   11|
+----------+-----+
only showing top 20 rows



In [146]:
# contando os valores area_util:

data.select('area_util').groupBy('area_util').count().show()

+---------+-----+
|area_util|count|
+---------+-----+
|      296|   33|
|      675|    1|
|      125|  287|
|      800|    5|
|      944|    2|
|       51|  273|
|      124|  174|
|      447|    1|
|      307|   29|
|      205|   49|
|      169|   90|
|      334|    6|
|      544|    1|
|      740|    1|
|      272|   11|
|      470|   11|
|      462|    2|
|       54|  414|
|      232|   41|
|      234|   37|
+---------+-----+
only showing top 20 rows



In [147]:
# apesar de existirem vários valores que se repetem, visualizando-os de maneira mais ampla vemos que estes são abrangentes e não podem ser descartados.

In [148]:
# para contar os valores nulos nas colunas area_total e area_util temos que:

from pyspark.sql.functions import col, isnan, when, count, lit

null_area = data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.select(['area_total', 'area_util']).columns])
null_area.show()

+----------+---------+
|area_total|area_util|
+----------+---------+
|      9194|        0|
+----------+---------+



In [149]:
# apenas a coluna area_total possui dados nulos.

# Convertendo os tipos das colunas

In [150]:
from pyspark.sql.types import IntegerType, DoubleType

In [151]:
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- valor: string (nullable = true)



## Para o tipo Integer

Coluna andar

In [152]:
data = data.withColumn('andar', data['andar'].cast(IntegerType()))

Coluna banheiros

In [153]:
data = data.withColumn('banheiros', data['banheiros'].cast(IntegerType()))

Coluna suites

In [154]:
data = data.withColumn('suites', data['suites'].cast(IntegerType()))

Coluna quartos

In [155]:
data = data.withColumn('quartos', data['quartos'].cast(IntegerType()))

## Para o tipo Double

Coluna area_total

In [156]:
data = data.withColumn('area_total', data['area_total'].cast(DoubleType()))

Coluna area_util

In [157]:
data = data.withColumn('area_util', data['area_util'].cast(DoubleType()))

Coluna condominio

In [158]:
data = data.withColumn('condominio', data['condominio'].cast(DoubleType()))

Coluna iptu

In [159]:
data = data.withColumn('iptu', data['iptu'].cast(DoubleType()))

Coluna valor

In [160]:
data = data.withColumn('valor', data['valor'].cast(DoubleType()))

In [161]:
# por fim, mostra-se o Schema dos dados para verificar se houve a conversão dos tipos dos dados das colunas:

data.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_total: double (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)



# Tratamento da coluna características

In [162]:
import pyspark.sql.functions as F

In [163]:
# os dados em que a coluna caracteristicas apresentan arrays vazios:

data.filter(F.size('caracteristicas') == 0).show(5)

+--------------------+-----+----------+---------+---------+---------------+-------+------+----+------------+------------+----------+------+--------+
|                  id|andar|area_total|area_util|banheiros|caracteristicas|quartos|suites|vaga|      bairro|        zona|condominio|  iptu|   valor|
+--------------------+-----+----------+---------+---------+---------------+-------+------+----+------------+------------+----------+------+--------+
|1fe78d41-b8e0-4d2...|    0|      44.0|     44.0|        1|             []|      2|     0|   0|       Irajá|  Zona Norte|     170.0|   0.0|110000.0|
|1fa1c1e5-e98c-433...|    4|      null|     55.0|        1|             []|      2|     0|   1|      Cosmos|  Zona Oeste|      null|  null|115000.0|
|a6ab01ae-3d40-40e...|    2|      null|     55.0|        1|             []|      2|     0|   0|Tomás Coelho|  Zona Norte|      null|  null|115000.0|
|1d8ff7a2-16bd-4e3...|    0|      null|     50.0|        1|             []|      1|  null|   0|      Centr

In [164]:
# São ao todo:

f'{data.filter(F.size("caracteristicas") == 0).count()} linhas do dataframe com essa especificidade'

'12736 linhas do dataframe com essa especificidade'

In [165]:
# substitui-se os arrays vazios por valores nulos:

data = data.withColumn('caracteristicas', F.when((F.size(F.col('caracteristicas')) == 0), F.lit(None)).otherwise(F.col('caracteristicas')))

In [166]:
# e verifica-se que o número de linhas da coluna caracteristicas com valores nulos é igual ao número de linhas que tinham arrays vazios:

data.filter(data['caracteristicas'].isNull()).count()

12736

In [167]:
data.filter(data['caracteristicas'].isNull()).show(5)

+--------------------+-----+----------+---------+---------+---------------+-------+------+----+------------+------------+----------+------+--------+
|                  id|andar|area_total|area_util|banheiros|caracteristicas|quartos|suites|vaga|      bairro|        zona|condominio|  iptu|   valor|
+--------------------+-----+----------+---------+---------+---------------+-------+------+----+------------+------------+----------+------+--------+
|1fe78d41-b8e0-4d2...|    0|      44.0|     44.0|        1|           null|      2|     0|   0|       Irajá|  Zona Norte|     170.0|   0.0|110000.0|
|1fa1c1e5-e98c-433...|    4|      null|     55.0|        1|           null|      2|     0|   1|      Cosmos|  Zona Oeste|      null|  null|115000.0|
|a6ab01ae-3d40-40e...|    2|      null|     55.0|        1|           null|      2|     0|   0|Tomás Coelho|  Zona Norte|      null|  null|115000.0|
|1d8ff7a2-16bd-4e3...|    0|      null|     50.0|        1|           null|      1|  null|   0|      Centr